In [12]:
nltk.download('vader_lexicon') # get lexicons data
nltk.download('punkt') # for tokenizer
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to C:\Users\Su
[nltk_data]     Yunhua\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Su
[nltk_data]     Yunhua\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Su
[nltk_data]     Yunhua\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [1]:
import pandas as pd
import numpy as np

# misc
import datetime as dt
from pprint import pprint
from itertools import chain



# sentiment analysis
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, RegexpTokenizer # tokenize words
from nltk.corpus import stopwords

# visualization
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 8) # default plot size
import seaborn as sns
sns.set(style='whitegrid', palette='Dark2')
from wordcloud import WordCloud

In [2]:
dt = pd.read_csv('Bitcoin 100020230306.csv')
dt

,no.,url,date,author,score,flair,num_comments,title,body,comments
0,1,https://v.redd.it/e7io27rdgt001,2017-11-28,LeeWallis,48612,/r/all,4435,"It's official! 1 Bitcoin = $10,000 USD",NaN,Where’s the guy that’s going to eat his left n...
1,2,https://v.redd.it/typ8fdslz3e01,2018-02-03,Tricky_Troll,48537,/r/all,763,The last 3 months in 47 seconds.,NaN,"Brilliant.\r\n\r\nEdit: \r\n\r\nWell, as I’m h..."
2,3,https://v.redd.it/gazga81l29g61,2021-02-08,AlphaGrayWolf,45542,/r/all,2876,Tesla buys $1.5b in Bitcoin and is looking to ...,NaN,Complete segment here: https://www.youtube.com...
3,4,https://i.imgur.com/jyoZGyW.gifv,2017-11-25,wolfwolfz,42451,/r/all,3081,It's over 9000!!!,NaN,Welcome /r/all!\r\n\r\nEdit: **9000** upvotes!...
4,5,http://cdn.mutually.com/wp-content/uploads/201...,2018-01-06,NaN,42080,/r/all,768,Everyone who's trading BTC right now,NaN,I don't even bother giving advice anymore. Ins...
...,...,...,...,...,...,...,...,...,...,...
986,987,https://i.redd.it/t67o83zrim431.jpg,2019-06-15,TidusxX420,3026,NaN,221,Let’s try this again. It’s over 9000!!!,NaN,1 million by the end of the year or I eat my o...
987,988,https://i.redd.it/edv2oufrq7821.png,2019-01-03,helloluis,3016,NaN,109,"Happy 10th, Bitcoin!",NaN,We are now past 18:15:05 GMT when the Genesis ...
988,989,https://v.redd.it/2lwgri4f9z581,2021-12-16,Kashpantz,3022,NaN,408,So mainstream it is.... The words HODL and the...,NaN,"Awesome episode. For my sins, I let SP pass me..."
989,990,https://v.redd.it/2wxz5ucpryn71,2021-09-16,simplelifestyle,3018,NaN,216,Huge crowd turns up to see Hungary’s statue of...,NaN,Who paid for it? Shady Super Coder hoodie lmao...


In [6]:
dt_sum_num_comments = dt.groupby("date")['num_comments'].agg(sum)

In [8]:
dt_sum_num_comments = pd.DataFrame(dt_sum_num_comments).sort_values(by = 'date', ascending = True)
dt_sum_num_comments

,num_comments
date,
2013-11-19,834
2013-11-24,283
2014-02-13,765
2014-02-18,496
2014-02-26,1418
...,...
2022-12-31,339
2023-01-03,258
2023-01-08,176


In [9]:
dt_agg_list = dt.groupby('date')['title'].agg(list)
dt_agg_list = pd.DataFrame(dt_agg_list).sort_values(by = 'date', ascending = True)
dt_agg_list

,title
date,
2013-11-19,[I’m one of the Senators attending today's U.S...
2013-11-24,"[I was bored so I animated the ""Bitcoin Accept..."
2014-02-13,[on r/bitcoin right now]
2014-02-18,[Bitcoin takes a walk with Dogecoin]
2014-02-26,[Open Letter to Michael Casey - WSJ reporter w...
...,...
2022-12-31,[Finally!!]
2023-01-03,[Bitcoin vs The Fed]
2023-01-08,[Mr Beast learns a valuable lesson]


In [10]:
df = pd.merge(dt_sum_num_comments, dt_agg_list, on='date')

In [11]:
df

,num_comments,title
date,,
2013-11-19,834,[I’m one of the Senators attending today's U.S...
2013-11-24,283,"[I was bored so I animated the ""Bitcoin Accept..."
2014-02-13,765,[on r/bitcoin right now]
2014-02-18,496,[Bitcoin takes a walk with Dogecoin]
2014-02-26,1418,[Open Letter to Michael Casey - WSJ reporter w...
...,...,...
2022-12-31,339,[Finally!!]
2023-01-03,258,[Bitcoin vs The Fed]
2023-01-08,176,[Mr Beast learns a valuable lesson]


In [17]:
dt_agg_list_COMMENT = dt.groupby('date')['comments'].agg(list)
dt_agg_list_COMMENT = pd.DataFrame(dt_agg_list_COMMENT).sort_values(by = 'date', ascending = True)
dt_agg_list_COMMENT

,comments
date,
2013-11-19,[Thanks everyone. I appreciate your comments. ...
2013-11-24,[It is nicely done and it's great that someone...
2014-02-13,[http://i.imgur.com/MBlS7Wr.gif\r\n\r\nHigher ...
2014-02-18,"[Being neutral to dogecoin, this is pretty dar..."
2014-02-26,"[Sorry, all Michael Casey saw was\r\n\r\n**""He..."
...,...
2022-12-31,[Congrats and ignore any new DMs in 3-2-1. If ...
2023-01-03,[You can track which address BTC was sent to. ...
2023-01-08,[Fun fact: they steal the blanket to cover up ...


In [21]:
df = df.merge(dt_agg_list_COMMENT,on='date')

In [22]:
df

,num_comments,title,comments
date,,,
2013-11-19,834,[I’m one of the Senators attending today's U.S...,[Thanks everyone. I appreciate your comments. ...
2013-11-24,283,"[I was bored so I animated the ""Bitcoin Accept...",[It is nicely done and it's great that someone...
2014-02-13,765,[on r/bitcoin right now],[http://i.imgur.com/MBlS7Wr.gif\r\n\r\nHigher ...
2014-02-18,496,[Bitcoin takes a walk with Dogecoin],"[Being neutral to dogecoin, this is pretty dar..."
2014-02-26,1418,[Open Letter to Michael Casey - WSJ reporter w...,"[Sorry, all Michael Casey saw was\r\n\r\n**""He..."
...,...,...,...
2022-12-31,339,[Finally!!],[Congrats and ignore any new DMs in 3-2-1. If ...
2023-01-03,258,[Bitcoin vs The Fed],[You can track which address BTC was sent to. ...
2023-01-08,176,[Mr Beast learns a valuable lesson],[Fun fact: they steal the blanket to cover up ...


In [23]:
sia = SentimentIntensityAnalyzer()

In [25]:
def get_average_sentiment(comments_list):
    sentiment_sum = 0
    num_comments = len(comments_list)

    for comment in comments_list:
        sentiment_score = sia.polarity_scores(comment)['compound']
        sentiment_sum += sentiment_score

    return sentiment_sum / num_comments if num_comments != 0 else 0

df['average_sentiment'] = df['comments'].apply(get_average_sentiment)

In [26]:
df['average_sentiment_title'] = df['title'].apply(get_average_sentiment)

In [27]:
df

,num_comments,title,comments,average_sentiment,average_sentiment_title
date,,,,,
2013-11-19,834,[I’m one of the Senators attending today's U.S...,[Thanks everyone. I appreciate your comments. ...,1.0000,0.3612
2013-11-24,283,"[I was bored so I animated the ""Bitcoin Accept...",[It is nicely done and it's great that someone...,1.0000,0.0000
2014-02-13,765,[on r/bitcoin right now],[http://i.imgur.com/MBlS7Wr.gif\r\n\r\nHigher ...,1.0000,0.0000
2014-02-18,496,[Bitcoin takes a walk with Dogecoin],"[Being neutral to dogecoin, this is pretty dar...",1.0000,0.0000
2014-02-26,1418,[Open Letter to Michael Casey - WSJ reporter w...,"[Sorry, all Michael Casey saw was\r\n\r\n**""He...",1.0000,0.0000
...,...,...,...,...,...
2022-12-31,339,[Finally!!],[Congrats and ignore any new DMs in 3-2-1. If ...,1.0000,0.0000
2023-01-03,258,[Bitcoin vs The Fed],[You can track which address BTC was sent to. ...,1.0000,0.0000
2023-01-08,176,[Mr Beast learns a valuable lesson],[Fun fact: they steal the blanket to cover up ...,0.9974,0.4767


In [15]:
#bitcoin=titles of all post
print(len(dt['title']))

991
